# Calculate average path propagation/attenuation according to ITU-R P.452 (16)

## License

```
Calculate path propagation/attenuation according to ITU-R P.452 (16)
in presence of geography and under flat-earth assumption and comparison.
Copyright (C) 2021+ Gyula Jozsa (gjozsa@mipfr.de) 
& Benjamin Winkel (bwinkel@mpifr.de)

This program is free software; you can redistribute it and/or
modify it under the terms of the GNU General Public License
as published by the Free Software Foundation; either version 2
of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301, USA.
```

In [ ]:
%matplotlib inline

In [ ]:
from collections import namedtuple, OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from pycraf import pathprof
from pycraf import conversions as cnv
from astropy import units as u
from astropy.utils.console import ProgressBar
import os.path
import copy
import time

With pycraf it is very easy to query SRTM data for arbitrary positions. However, you will need to have to download such SRTM data, either in advance or during your session. Please see the [documentation](https://bwinkel.github.io/pycraf/pathprof/working_with_srtm.html) for more details.

Use the following, if you already have the SRTMDATA environment variable set, but not downloaded any ".hgt" files so far.

In [ ]:
# pathprof.SrtmConf.set(download='missing', server='viewpano')

If you neither have the environment variable set nor ".hgt" files downloaded (you can use any target directory, of course):

In [ ]:
# pathprof.SrtmConf.set(download='missing', server='viewpano', srtm_dir='.')

If you do not have the environment variable set but you have all .hgt files downloaded:

In [ ]:
pathprof.SrtmConf.set(srtm_dir='/Users/jozsa/software/srtm/all')

## Averaging path propagation loss maps

Often it is desired to generate maps of path attenuation values, e.g.,
to quickly determine regions where the necessary separations between a
potential interferer and the victim terminal would be too small,
potentially leading to radio frequency interference (RFI). In this
example we use attenuation maps to average the attenuation at a
certain distance from the over all directions, for a set of telescopes
and a set of frequencies. We will use our knowledge from notebook 03c
to do so. We first encapsule the functionality in some functions and
objects.

### The backbone information
We create a lookup table of telescopes (i.e. a dictionary), in which
we encapsule their names, geographical position, height above the
ground, and their operating frequencies. We also create a dictionary
of landmarks, for orientation.

In [ ]:
Telescope = namedtuple('telescope',
                       ['name', 'coordinates', 'height', 'frequencies'])
telinfo = {
    'Wb': Telescope('RT Westerbork', (6.60334 * u.deg, 52.91474 * u.deg),
                    22.5 * u.m, (610. * u.MHz, 6650. * u.MHz)),
    'Eb': Telescope('RT Effelsberg', (6.88361 * u.deg, 50.52483 * u.deg),
                    50. * u.m, (610. * u.MHz, 6650. * u.MHz))
}

Site = namedtuple('site', ['name', 'coord', 'pixoff', 'color'])
sites = OrderedDict([
                     # ID, tuple(Name,
                     #           (lon, lat),
                     #           (xoff, yoff),
                     #           color
                     #            )
                     ('RT Effelsberg (Eb)',
                      Site('RT Effelsberg (Eb)',
                           (6.88361, 50.52483),
                           (20, +30),
                           'k')
                      ),
                     ('Cologne',
                      Site('Cologne',
                           (6.9602, 50.9377),
                           (20, +30),
                           'k')
                      ),
                     ('Bonn',
                      Site('Bonn',
                           (7.0982, 50.7375),
                           (20, +30),
                           'k')
                      ),
                     ('RT Westerbork (Wb)',
                      Site('RT Westerbork (Wb)',
                           (6.60334, 52.91474),
                           (20, +30),
                           'k')
                      ),
                     ('Assen',
                      Site('Assen',
                           (6.5625, 52.99667),
                           (20, +30),
                           'k')
                      ),
                     ('Groningen',
                      Site('Groningen',
                           (6.56667, 53.21917),
                           (20, +30),
                           'k')
                      ),
                     ('Istanbul',
                      Site('Istanbul',
                           (28.955, 41.013611),
                           (20, +30),
                           'k')
                      ),
                     ('London',
                      Site('London',
                           (0.1275, 51.507222),
                           (20, +30),
                           'k')
                      ),
                     ('Saint Petersburg',
                      Site('Saint Petersburg',
                           (30.3, 59.95),
                           (20, +30),
                           'k')
                      ),
                     ('Berlin',
                      Site('Berlin',
                           (13.383333, 52.516667),
                           (20, +30),
                           'k')
                      ),
                     ('Madrid',
                      Site('Madrid',
                           (3.716667, 40.383333),
                           (20, +30),
                           'k')
                      ),
                     ('Kiev',
                      Site('Kiev',
                           (30.523333, 50.45),
                           (20, +30),
                           'k')
                      ),
                     ('Rome',
                      Site('Rome',
                           (12.5, 41.9),
                           (20, +30),
                           'k')
                      ),
                     ('Paris',
                      Site('Paris',
                           (2.3508, 48.8567),
                           (20, +30),
                           'k')
                      ),
                     ('Bucharest',
                      Site('Bucharest',
                           (26.103889, 44.4325),
                           (20, +30),
                           'k')
                      ),
                     ('Minsk',
                      Site('Minsk',
                           (27.566667, 53.9),
                           (20, +30),
                           'k')
                      ),
                     ('Vienna',
                      Site('Vienna',
                           (16.366667, 48.2),
                           (20, +30),
                           'k')
                      ),
                     ('Hamburg',
                      Site('Hamburg',
                           (10.001389, 53.565278),
                           (20, +30),
                           'k')
                      ),
                     ('Budapest',
                      Site('Budapest',
                           (19.051389, 47.4925),
                           (20, +30),
                           'k')
                      ),
                     ('Warsaw',
                      Site('Warsaw',
                           (21.016667, 52.233333),
                           (20, +30),
                           'k')
                      ),
                     ('Barcelona',
                      Site('Barcelona',
                           (2.183333, 41.383333),
                           (20, +30),
                           'k')
                      ),
                     ('Munich',
                      Site('Munich',
                           (11.566667, 48.133333),
                           (20, +30),
                           'k')
                      ),
                     ('Kharkiv',
                      Site('Kharkiv',
                           (36.231389, 50.004444),
                           (20, +30),
                           'k')
                      ),
                     ('Milan',
                      Site('Milan',
                           (9.183333, 45.466667),
                           (20, +30),
                           'k')
                      ),
                     ('Prague',
                      Site('Prague',
                           (14.416667, 50.083333),
                           (20, +30),
                           'k')
                      ),
                     ('Nizhny Novgorod',
                      Site('Nizhny Novgorod',
                           (44.0075, 56.326944),
                           (20, +30),
                           'k')
                      ),
                     ('Sofia',
                      Site('Sofia',
                           (23.33, 42.7),
                           (20, +30),
                           'k')
                      ),
                     ('Kazan',
                      Site('Kazan',
                           (49.134722, 55.790278),
                           (20, +30),
                           'k')
                      ),
                     ('Brussels',
                      Site('Brussels',
                           (4.35, 50.85),
                           (20, +30),
                           'k')
                      ),
                     ('Samara',
                      Site('Samara',
                           (50.140833, 53.202778),
                           (20, +30),
                           'k')
                      ),
                     ('Belgrade',
                      Site('Belgrade',
                           (20.466667, 44.816667),
                           (20, +30),
                           'k')
                      ),
                     ('Rostov-on-Don',
                      Site('Rostov-on-Don',
                           (39.7, 47.233333),
                           (20, +30),
                           'k')
                      ),
                     ('Birmingham',
                      Site('Birmingham',
                           (1.893611, 52.483056),
                           (20, +30),
                           'k')
                      ),
                     ('Ufa',
                      Site('Ufa',
                           (55.966667, 54.75),
                           (20, +30),
                           'k')
                      ),
                     ('Cologne',
                      Site('Cologne',
                           (6.952778, 50.936389),
                           (20, +30),
                           'k')
                      ),
                     ('Perm',
                      Site('Perm',
                           (56.316667, 58),
                           (20, +30),
                           'k')
                      ),
                     ('Voronezh',
                      Site('Voronezh',
                           (39.210556, 51.671667),
                           (20, +30),
                           'k')
                      ),
                     ('Volgograd',
                      Site('Volgograd',
                           (44.516667, 48.7),
                           (20, +30),
                           'k')
                      ),
                     ('Odessa',
                      Site('Odessa',
                           (30.733333, 46.466667),
                           (20, +30),
                           'k')
                      ),
                     ('Naples',
                      Site('Naples',
                           (14.25, 40.833333),
                           (20, +30),
                           'k')
                      ),
                     ('Dnipropetrovsk',
                      Site('Dnipropetrovsk',
                           (34.983333, 48.45),
                           (20, +30),
                           'k')
                      ),
                     ('Mariehamn',
                      Site('Mariehamn',
                           (19.9, 60.116667),
                           (20, +30),
                           'k')
                      ),
                     ('Tirana',
                      Site('Tirana',
                           (19.816667, 41.31666667),
                           (20, +30),
                           'k')
                      ),
                     ('Andorra la Vella',
                      Site('Andorra la Vella',
                           (1.516667, 42.5),
                           (20, +30),
                           'k')
                      ),
                     ('Yerevan',
                      Site('Yerevan',
                           (44.5, 40.16666667),
                           (20, +30),
                           'k')
                      ),
                     ('Baku',
                      Site('Baku',
                           (49.866667, 40.38333333),
                           (20, +30),
                           'k')
                      ),
                     ('Sarajevo',
                      Site('Sarajevo',
                           (18.416667, 43.86666667),
                           (20, +30),
                           'k')
                      ),
                     ('Zagreb',
                      Site('Zagreb',
                           (16, 45.8),
                           (20, +30),
                           'k')
                      ),
                     ('Nicosia',
                      Site('Nicosia',
                           (33.366667, 35.16666667),
                           (20, +30),
                           'k')
                      ),
                     ('Copenhagen',
                      Site('Copenhagen',
                           (12.583333, 55.66666667),
                           (20, +30),
                           'k')
                      ),
                     ('Tallinn',
                      Site('Tallinn',
                           (24.716667, 59.43333333),
                           (20, +30),
                           'k')
                      ),
                     ('Torshavn',
                      Site('Torshavn',
                           (-6.766667, 62),
                           (20, +30),
                           'k')
                      ),
                     ('Helsinki',
                      Site('Helsinki',
                           (24.933333, 60.16666667),
                           (20, +30),
                           'k')
                      ),
                     ('Tbilisi',
                      Site('Tbilisi',
                           (44.833333, 41.68333333),
                           (20, +30),
                           'k')
                      ),
                     ('Gibraltar',
                      Site('Gibraltar',
                           (-5.35, 36.13333333),
                           (20, +30),
                           'k')
                      ),
                     ('Athens',
                      Site('Athens',
                           (23.733333, 37.98333333),
                           (20, +30),
                           'k')
                      ),
                     ('Saint Peter Port',
                      Site('Saint Peter Port',
                           (-2.533333, 49.45),
                           (20, +30),
                           'k')
                      ),
                     ('Reykjavik',
                      Site('Reykjavik',
                           (-21.95, 64.15),
                           (20, +30),
                           'k')
                      ),
                     ('Dublin',
                      Site('Dublin',
                           (-6.233333, 53.31666667),
                           (20, +30),
                           'k')
                      ),
                     ('Douglas',
                      Site('Douglas',
                           (-4.483333, 54.15),
                           (20, +30),
                           'k')
                      ),
                     ('Saint Helier',
                      Site('Saint Helier',
                           (-2.1, 49.18333333),
                           (20, +30),
                           'k')
                      ),
                     ('Pristina',
                      Site('Pristina',
                           (21.166667, 42.66666667),
                           (20, +30),
                           'k')
                      ),
                     ('Riga',
                      Site('Riga',
                           (24.1, 56.95),
                           (20, +30),
                           'k')
                      ),
                     ('Vaduz',
                      Site('Vaduz',
                           (9.516667, 47.13333333),
                           (20, +30),
                           'k')
                      ),
                     ('Vilnius',
                      Site('Vilnius',
                           (25.316667, 54.68333333),
                           (20, +30),
                           'k')
                      ),
                     ('Luxembourg',
                      Site('Luxembourg',
                           (6.116667, 49.6),
                           (20, +30),
                           'k')
                      ),
                     ('Skopje',
                      Site('Skopje',
                           (21.433333, 42),
                           (20, +30),
                           'k')
                      ),
                     ('Valletta',
                      Site('Valletta',
                           (14.5, 35.88333333),
                           (20, +30),
                           'k')
                      ),
                     ('Chisinau',
                      Site('Chisinau',
                           (28.85, 47),
                           (20, +30),
                           'k')
                      ),
                     ('Monaco',
                      Site('Monaco',
                           (7.416667, 43.73333333),
                           (20, +30),
                           'k')
                      ),
                     ('Podgorica',
                      Site('Podgorica',
                           (19.266667, 42.43333333),
                           (20, +30),
                           'k')
                      ),
                     ('Amsterdam',
                      Site('Amsterdam',
                           (4.916667, 52.35),
                           (20, +30),
                           'k')
                      ),
                     ('North Nicosia',
                      Site('North Nicosia',
                           (33.366667, 35.183333),
                           (20, +30),
                           'k')
                      ),
                     ('Oslo',
                      Site('Oslo',
                           (10.75, 59.91666667),
                           (20, +30),
                           'k')
                      ),
                     ('Lisbon',
                      Site('Lisbon',
                           (-9.133333, 38.71666667),
                           (20, +30),
                           'k')
                      ),
                     ('Moscow',
                      Site('Moscow',
                           (37.6, 55.75),
                           (20, +30),
                           'k')
                      ),
                     ('San Marino',
                      Site('San Marino',
                           (12.416667, 43.93333333),
                           (20, +30),
                           'k')
                      ),
                     ('Bratislava',
                      Site('Bratislava',
                           (17.116667, 48.15),
                           (20, +30),
                           'k')
                      ),
                     ('Ljubljana',
                      Site('Ljubljana',
                           (14.516667, 46.05),
                           (20, +30),
                           'k')
                      ),
                     ('Longyearbyen',
                      Site('Longyearbyen',
                           (15.633333, 78.21666667),
                           (20, +30),
                           'k')
                      ),
                     ('Stockholm',
                      Site('Stockholm',
                           (18.05, 59.33333333),
                           (20, +30),
                           'k')
                      ),
                     ('Bern',
                      Site('Bern',
                           (7.466667, 46.91666667),
                           (20, +30),
                           'k')
                      ),
                     ('Ankara',
                      Site('Ankara',
                           (32.866667, 39.93333333),
                           (20, +30),
                           'k')
                      ),
                     ('Kyiv',
                      Site('Kyiv',
                           (30.516667, 50.43333333),
                           (20, +30),
                           'k')
                      ),
                     ('London',
                      Site('London',
                           (-0.083333, 51.5),
                           (20, +30),
                           'k')
                      ),
                     ('Vatican City',
                      Site('Vatican City',
                           (12.45, 41.9),
                           (20, +30),
                           'k')
                      )
                     ])


### Topographical information
We create a function that can generate a topographical map and a
hardcopy

In [ ]:
def plotheightmap(heightmap, sites, lons, lats, outname=None,
                  overwrite=True, title=''):
    """Plot map showing height above geoid (topographic map)

    Takes a numpy array heightmap, including an array of longitudes
    and latitudes and generates a topographic map. An additional input
    is an ordered dict of named tuples:

    ID: Site(Name,
             (lon, lat),
             (xoff, yoff),
             color
             )

    indicating landmarks (cities), which are pointed at by arrows and
    whose names are displayed on the plot. Name is the name of the
    site, (lon, lat), the longitude and latitude pair, (xoff, yoff)
    the offset of the name on the map, and color the (matplotlib)
    color of the name.

    The map can be exported to a file outname, overwriting any
    existing file if overwrite is True. Optionally, the map gets a
    title specified with the keyword title, placed on the top of the
    map.

    Parameters:
    heightmap (ndarray):   Height map im m above geoid,
                           as provided by function
                           pycraf.pathprof.srtm_height_map
    sites (OrderedDict):   Dictionary with site information
    lons (ndarray):        Longitudes,
                           as provided by function
                           pycraf.pathprof.srtm_height_map
    lats (ndarray):        Latitudes,
                           as provided by function
                           pycraf.pathprof.srtm_height_map
    outname (str or None): Name of output file
    overwrite (bool):      Overwrite existing output file? (True:yes)
    title (str):           Optional title of map

    Returns:
        None
    """
    _lons = lons.to(u.deg).value
    _lats = lats.to(u.deg).value
    _heightmap = heightmap.to(u.m).value

    vmin, vmax = -20, 950
    terrain_cmap, terrain_norm = pathprof.terrain_cmap_factory(
        sealevel=0.5, vmax=vmax)
    _heightmap[_heightmap < 0] = 0.51  # fix for coastal region

    # plt.close()

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_axes((0.1, 0.12, 0.88, 0.83))
    cbax = fig.add_axes((0.15, 0.145, 0.78, .02))
    # ax = fig.add_axes((0., 0., 1.0, 1.0))
    # cbax = fig.add_axes((0., 0., 1.0, .02))

    cim = ax.imshow(
        _heightmap,
        origin='lower', interpolation='nearest',
        cmap=terrain_cmap, norm=terrain_norm,
        # vmin=vmin, vmax=vmax,  # deprecated in newer matplotlib versions
        extent=(_lons[0], _lons[-1], _lats[0], _lats[-1]),
        )
    cbar = fig.colorbar(
        cim, cax=cbax, orientation='horizontal'
    )
    aspect = abs(_lons[-1] - _lons[0]) / abs(_lats[-1] - _lats[0])
    ax.set_aspect(aspect)
    cbar.set_label(r'Height (amsl)', color='k')
    for t in cbax.xaxis.get_major_ticks():
        t.tick1line.set_visible(True)
        t.tick2line.set_visible(True)
        t.label1.set_visible(False)
        t.label2.set_visible(True)
    ctics = np.arange(0, 1350, 200)
    cbar.set_ticks(ctics)
    ctics = cbar.get_ticks()
    cbar.ax.set_xticklabels(map('{:.0f} m'.format, ctics), color='k')
    ax.set_xlabel('Longitude [deg]')
    ax.set_ylabel('Latitude [deg]')

    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.set_title(title, y=0.95, color='k', size=16)

    for site_id, site in sites.items():
        color = site.color
        # color = 'k'
        ax.annotate(
            site.name, xy=site.coord, xytext=site.pixoff,
            textcoords='offset points', color=color,
            arrowprops=dict(arrowstyle="->", color=color)
            )

    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    # plt.show()
    if isinstance(outname, type('')):
        if overwrite:
            if os.path.isfile(outname):
                os.remove(outname)
        plt.savefig(outname)

    return

### Plotting an attenuation map
We saw how to generate and plot the attenuation map in notebook 3c.
Here, we create a plotting funtion to do so.

In [ ]:
def plot_attenmap(total_atten, sites, lons, lats, outname=None,
                  overwrite=True, vmin=-5, vmax=175, title='',
                  ctics=20):
    """Plot map showing attenuation map

    Takes a pycraf array total_atten (attenuation in dB), an array of
    longitudes and an array of latitudes and generates a topographic
    map. An additional input is an ordered dict of named tuples:

    ID: Site(Name,
             (lon, lat),
             (xoff, yoff),
             color
             )

    indicating landmarks (cities), which are pointed at by arrows and
    whose names are displayed on the plot. Name is the name of the
    site, (lon, lat), the longitude and latitude pair, (xoff, yoff)
    the offset of the name on the map, and color the (matplotlib)
    color of the name.

    The map can be exported to a file outname, overwriting any
    existing file if overwrite is True. vmin and vmax indicate maximum
    and minimum of the color scheme. Optionally, the map gets a title
    specified with the keyword title, placed on the top of the
    map. The parameter ctics determines the difference (in dB) between
    tics on the color bar.

    Parameters:
    heightmap (ndarray):   Height map im m above geoid,
                           as provided by function
                           pycraf.pathprof.srtm_height_map
    sites (OrderedDict):   Dictionary with site information
    lons (ndarray):        Longitudes,
                           as provided by function
                           pycraf.pathprof.srtm_height_map
    lats (ndarray):        Latitudes,
                           as provided by function
                           pycraf.pathprof.srtm_height_map
    outname (str or None): Name of output file
    overwrite (bool):      Overwrite existing output file (True:yes)?
    vmin (float):          Minimum of colour scheme
    vmax (float):          Maximum of colour scheme
    title (str):           Optional title of map
    ctics (float):         Difference between tickmarks on colour bar

    Returns:
        None
    """
    _lons = lons.to(u.deg).value
    _lats = lats.to(u.deg).value

    # plt.close()

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_axes((0.1, 0.08, 0.88, 0.87))
    cbax = fig.add_axes((0.15, 0.1, 0.78, .02))
    cim = ax.imshow(
        total_atten.to(cnv.dB).value,
        origin='lower', interpolation='nearest', cmap='inferno_r',
        vmin=vmin, vmax=vmax,
        extent=(_lons[0], _lons[-1], _lats[0], _lats[-1]),
        )
    cbar = fig.colorbar(
        cim, cax=cbax, orientation='horizontal'
        )
    ax.set_aspect(abs(_lons[-1] - _lons[0]) / abs(_lats[-1] - _lats[0]))
    cbar.set_label(r'Path propagation loss', color='w')
    cbax.xaxis.set_label_position('top')
    for t in cbax.xaxis.get_major_ticks():
        t.tick1line.set_visible(True)
        t.tick2line.set_visible(True)
        t.tick2line.set_color('w')
        t.label1.set_visible(False)
        t.label2.set_visible(True)

    ctics = np.arange(0, 300, ctics)
    cbar.set_ticks(ctics)
    ctics = cbar.get_ticks()
    cbar.ax.set_xticklabels(map('{:.0f} dB'.format, ctics), color='w')
    ax.set_xlabel('Longitude [deg]')
    ax.set_ylabel('Latitude [deg]')

    for site_id, site in sites.items():
        color = site.color
        color = 'g'
        ax.annotate(
            site.name, xy=site.coord, xytext=site.pixoff,
            textcoords='offset points', color=color,
            arrowprops=dict(arrowstyle="->", color=color)
            )

    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.set_title(title, y=0.95, color='w', size=16)
    if isinstance(outname, type('')):
        if overwrite:
            if os.path.isfile(outname):
                os.remove(outname)
        plt.savefig(outname)

### Statistics along a circle
Given the attenuation map, we are now able to create statistics about
the attenuation. Often, we do not need to predict the attenuation at a
specific geographical point, but we want to know, what the statistics
of the attenuation is at a certain distance from the receiving
antenna, minimum, maximum, percentiles, median. Using pycraf structs,
we generate this type of statistics with the following function.

In [ ]:
def averatt(atten_map, hprof_cache, lo_perc=5., hi_perc=95., rmax=10*u.km,
            bins=100):
    """Return circular statistics of attenuation

    Given an attenuation map atten_map in addition with either a map
    providing the physical distances between the reference point (at
    which attenuation = 0) and the single pixels or the output of a
    run of the pycraf.pathprof.height_map_data method, the function
    creates statistics along concentric rings (with a constant geoid
    distance to the centre), which are returned as a list of numpy
    arrays. First, the distance between 0 and rmax is binned into bins
    equally-sized distance intervals. For the pixels in each distance
    interval, the following statistics is calculated for the
    attenuation map:

    - median
    - percentile with percentage given by lo_perc
    - percentile with percentage given by hi_perc
    - maximum
    - minimum

    In addition the average distance for each bin is calculated. Each
    statistics is converted to a numpy array.  The function returns a
    dictionary with the following arrays for the given keys:

    'radius': average radii
    'median': median at each interval
    'perc_lo': percentile for lo_perc percentage at each interval
    'perc_hi': percentile for hi_perc percentage at each interval
    'minimum': minimum for each interval
    'maximum': maximum for each interval

    To e.g. study the median of the attenuation with radius, one can
    use the 'radius' array and the 'median' array etc.

    Arguments:
    atten_map (pycraf array): Attenuation map
    hprof_cache (numpy array or pycraf height profile object): map
        with physical distances between reference point and pixel
    lo_perc (float): Lower percentage for the calculation of lower
        percentile
    hi_perc (float): Higher percentage for the calculation of upper
        percentile
    rmax (float, int, or astropy length): maximum radius (in km for
        int or float)
    bins (int): number of bins

    Return:
    dictionary with statistics vs. radius.
    """

    # Distance map in km
    if (isinstance(hprof_cache, type(np.zeros((1, 1))))):
        dis = hprof_cache
    else:
        dis = hprof_cache['dist_map']

    # Flatten maps
    # allatt = _total_atten.reshape(dis.shape[0]*dis.shape[1]).to(cnv.dimless)
    # allatt = np.power(10., _total_atten.reshape(
    #                   dis.shape[0]*dis.shape[1]).value/10.)
    allatt = atten_map.reshape(dis.shape[0]*dis.shape[1]).value
    alldis = dis.reshape(dis.shape[0]*dis.shape[1])

    # Create bins
    if isinstance(rmax, type(float)) or isinstance(rmax, type(int)):
        rmaxi = rmax
    else:
        rmaxi = rmax.to(u.km).value
    binsar = np.linspace(0, rmaxi, bins+1)

    # Get stats
    digitized = np.digitize(alldis, binsar)

    averatt = {}
    averatt['bin_dists'] = \
        np.array([alldis[digitized == i].mean()
                  for i in range(1, len(binsar))])
    averatt['bin_lo'] = \
        np.array([np.percentile(allatt[digitized == i], lo_perc)
                  for i in range(1, len(binsar))])
    averatt['bin_med'] = \
        np.array([np.percentile(allatt[digitized == i], 50)
                  for i in range(1, len(binsar))])
    averatt['bin_hi'] = \
        np.array([np.percentile(allatt[digitized == i], hi_perc)
                  for i in range(1, len(binsar))])
    averatt['bin_min'] = \
        np.array([allatt[digitized == i].min()
                  for i in range(1, len(binsar))])
    averatt['bin_max'] = \
        np.array([allatt[digitized == i].max()
                  for i in range(1, len(binsar))])

    return averatt

### Plotting statistical profiles
Now that we generated the statistics of the attenuation as a function
of radius but averaged over equidistant rings, we can plot the
results. The following function is able to generate an average
attenuation profile, and, using an inlay, another plot showing the
difference to an approximation of a flat earth.

In [ ]:
def plotaveratt(averatts, title=None, outname=None,
                overwrite=True, ymin=100, ymax=200, averatts2=None,
                inlay=(0.3, 0.13, 0.65, 0.25), avmd=2, grid=10):
    """Plot circularly averaged attenuations with radius

    Given the output of function averatt (attenuation statistics along
    concentric rings), plots median as a black line with green area
    for upper and lower percentile, and cyan area between min and max
    for each radius. If outname is not None, a file with the viewgraph
    will be generated. If overwrite is True any existing file with the
    same name will be overwritten. ymin and ymax are the minimimum and
    the maximum of the viewgraph. If averatts2 is not None, produces
    red line showing the median in averatts2, and an inlay with
    dimensions specified by the inlay parameter, a list or tuple with
    the position of the lower left corner given by the first two
    members, and width and height relative to the plot widths and
    height. The inlay shows above quantities of averatts minus the
    median in averatts2. Instead of the average, inlay contains an
    vertical line indicating the average difference between the
    medians in averatts and averatts2, starting at radius avmd (in
    km). Grid denotes the vertical grid line separation in the main
    viewgraph.

    Arguments:
    averatts (dict): Output of averatt function
    title (str or None): Title of the plot
    outname (str or None): Name of output file
    overwrite (bool): Overwrite existing output? (True: yes)
    ymin (float): Minimum on ordinate
    ymax (float): Maximum on ordinate
    averatts (dict): Output of averatt function (second dict)
    avmd (float): Minimum radius for averaging inlay curve
    inlay (tuple): Dimensions of inlay
    grid (float): Grid line separation in the main viewgraph

    Returns:
        None
    """

    # bin_means_dB = (cnv.dimless*bin_means).to(cnv.dB).value
    # bin_upper_dB = (cnv.dimless*(bin_means+bin_stdev)).to(cnv.dB).value
    # bin_lower_dB = (cnv.dimless*(bin_means-bin_stdev)).to(cnv.dB).value
    bin_dists = averatts['bin_dists']
    bin_med_dB = averatts['bin_med']
    bin_hi_dB = averatts['bin_hi']
    bin_lo_dB = averatts['bin_lo']
    bin_max_dB = averatts['bin_max']
    bin_min_dB = averatts['bin_min']

    # plt.close()
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_axes((0.1, 0.08, 0.88, 0.87))
    ax.plot(bin_dists, bin_med_dB, 'k-')
    ax.fill_between(bin_dists, bin_lo_dB, bin_hi_dB,
                    fc='g', ec='r', linewidth=0, linestyle='None')
    ax.fill_between(bin_dists, bin_lo_dB, bin_min_dB,
                    fc='cyan', linewidth=0, linestyle='None', alpha=0.5)
    ax.fill_between(bin_dists, bin_hi_dB, bin_max_dB,
                    fc='cyan', linewidth=0, linestyle='None', alpha=0.5)
    ax.set_xlabel('Distance [km]', size=24)
    ax.set_ylabel('Average attenuation [dB]', size=24)
    ax.xaxis.set_tick_params(labelsize=16)
    ax.yaxis.set_tick_params(labelsize=16)
    ax.set_ylim(ymin, ymax)
    if grid:
        ax.yaxis.grid()

    if isinstance(title, type('')):
        ax.set_title(title, size=24)

    if not isinstance(averatts2, type(None)):
        bin_med_dB = averatts2['bin_med']
        ax.plot(bin_dists, bin_med_dB, 'r-')
        if not isinstance(inlay, type(None)):
            ax = fig.add_axes(inlay)
            bin_diff_dB = averatts['bin_med']-bin_med_dB
            ad = np.average(bin_diff_dB[bin_dists > avmd])
            bin_adiff_dB = bin_diff_dB[bin_dists > avmd]*0+ad
            bin_hi_dBd = averatts['bin_hi']-bin_med_dB
            bin_lo_dBd = averatts['bin_lo']-bin_med_dB
            bin_max_dBd = averatts['bin_max']-bin_med_dB
            bin_min_dBd = averatts['bin_min']-bin_med_dB
            ax.plot(bin_dists, bin_diff_dB, 'k-')
            ax.plot(bin_dists[bin_dists > avmd], bin_adiff_dB, 'r-')
            ax.fill_between(bin_dists, bin_lo_dBd, bin_hi_dBd, fc='g',
                            ec='r', linewidth=0, linestyle='None')
            ax.fill_between(bin_dists, bin_lo_dBd, bin_min_dBd, fc='cyan',
                            linewidth=0, linestyle='None', alpha=0.5)
            ax.fill_between(bin_dists, bin_hi_dBd, bin_max_dBd, fc='cyan',
                            linewidth=0, linestyle='None', alpha=0.5)
            ax.set_ylabel('Difference [dB]', size=10)
            ax.xaxis.set_tick_params(labelsize=10)
            ax.yaxis.set_tick_params(labelsize=10)
            shift = (ax.get_ylim()[1]-ax.get_ylim()[0])*0.05
            ax.annotate('{:.0f} dB'.format(ad),
                        xy=(bin_dists[-1]*0.9, ad+shift), color='r')
            if grid:
                ax.yaxis.grid()

    if isinstance(outname, type('')):
        if overwrite:
            if os.path.isfile(outname):
                os.remove(outname)
        plt.savefig(outname)
    return

### Generating multiple maps and profiles
As laid out above, we want to create attenuation maps and average
profiles for several telescopes and multiple frequencies. The
following function generates those maps for a single telescope, but
multiple frequencies. It will also not only generate a map/profile
assuming normal geography, but also assuming a flat earth.

In [ ]:
def getattdata(telname, tellist, params, genparams, transmitters):
    """Batch production of topographic and attenuation maps and statistics

    For a telescope with telname the function constructs a
    topographical map, a set of attenuation maps, and radial
    statistics based on the attenuation maps.  tellist is a
    dictionary, of which one of the keys must match telname. The
    values are a named tuple containing the telescope name, a tuple
    with its coordinates (longitude and latitude in astropy units),
    its height above the ground in astropy units, and an iterable
    containg the frequencies at which the telescope operates in
    astropy units. The function identifies the telescope in question
    from tellist. params is a dictionary of parameters for the
    calculation of the attenuation map, see description of functions
    pycraf.pathprof.srtm_height_map, pycraf.pathprof.height_map_data,
    and pycraf.pathprof.atten_map_fast:

    key             description
    map_size_lon:   Size of map in geographical longitude in astropy units
    map_size_lat:   Size of map in geographical latitude in astropy units
    omega:          Fraction of path over sea (astropy units)
    temperature:    Temperature in astropy units
    pressure:       Air pressure in astropy units
    timepercent:    Time percentage in astropy units, see P.452 for explanation
    zone_t:         Clutter type for transmitter terminal
    zone_r:         Clutter type for receiver terminal
    map_resolution: Resolution of map in astropy units

    genparms is a dictionary with parameters specific to the
    generation of radial statistics, see function averatt in this
    module:
    key      description
    lo_perc: Lower percentage for the calculation of lower percentile
    hi_perc: Higher percentage for the calculation of upper percentile
    rmax:    Maximum radius (in km for int or float)
    bins:    Number of bins

    transmitters is a dictionary of transmitters, in which the keys
    are the transmitted frequencies in astropy units and the value the
    corresponding height above the ground of the transmitter.

    For the given telescope the function produces a topographical map,
    then for each frequency an attenuation map (see function
    pycraf.pathprof.atten_map_fast) and a radial statistics object
    (see function averatt in this module), to then create the
    attenuation map and corresponding statistics under the assumption
    of a flat geoid geometry ("flat earth"). The output is a
    dictionary:

    key         description
    telname     Telescope name
    frequencies Frequencies as listed for telescope
    lons        Longitudes along longitude axis
    lats        Latitudes along latitude axis
    heightmap   A topographical map (ndarray)
    distmap     A map showing the physical distance w.r.t. the central pixel
    attenmaps   List of attenuation maps, one per frequency
    attenmaps_0 List of attenuation maps under flat-earth approximation,
                one per frequency
    averatts    List of statistics objects as returned by averatt, one per
                frequency
    averatts_0  List of statistics objects as returned by averatt, one per
                frequency, flat earth approximation

    Arguments:
    telname (str):      Telescope name
    tellist (dict):     List of available telescopes
    params (dict):      Parameter list for pycraf functions
    genparams (dict):   Parameter list for averatt function (this module)
    transmitters(dict): Dictionary with frequencies and heights

    Returns:
        A dictionary (see description)
  """
    start_time_tot = time.time()
    output = {}
    output['telname'] = telname
    telpars = tellist[telname]
    output['frequencies'] = telpars.frequencies
    print('{}: Creating height map'.format(telname))
    start_time_loc = time.time()
    lons, lats, heightmap = pathprof.srtm_height_map(
        telpars.coordinates[0], telpars.coordinates[1],
        params['map_size_lon'], params['map_size_lat'],
        map_resolution=params['map_resolution']
        )
    print('Time after start: {} minutes'.format((time.time() -
                                                 start_time_tot)/60.))
    print('Time for this task: {} minutes'.format((time.time() -
                                                   start_time_loc)/60.))
    output['lons'] = lons
    output['lats'] = lats
    output['heightmap'] = heightmap
    print('{}: Creating terrain information map'.format(telname))
    start_time_loc = time.time()
    hprof_cache = pathprof.height_map_data(
        telpars.coordinates[0], telpars.coordinates[1],
        params['map_size_lon'], params['map_size_lat'],
        map_resolution=params['map_resolution'],
        zone_t=params['zone_t'], zone_r=params['zone_r'],
        )
    print('Time after start: {} minutes'.format((time.time() -
                                                 start_time_tot)/60.))
    print('Time for this task: {} minutes'.format((time.time() -
                                                   start_time_loc)/60.))
    output['distmap'] = hprof_cache['dist_map']
    attenmaps = []
    attenmaps_0 = []
    averatts = []
    averatts_0 = []
    for frequency in output['frequencies']:
        print('{}: Creating attenuation map at'.format(telname),
              ' {:.0f} MHz'.format(frequency.to(u.MHz).value))
        start_time_loc = time.time()

        h_rg = transmitters[frequency.to(u.MHz)]
        attenmaps += [pathprof.atten_map_fast(
            frequency,
            params['temperature'],
            params['pressure'],
            telpars.height,
            h_rg,
            params['timepercent'],
            hprof_cache
        )['L_b']]
        print('Time after start: {} minutes'.format((time.time() -
                                                     start_time_tot)/60.))
        print('Time for this task: {} minutes'.format((time.time() -
                                                       start_time_loc)/60.))

        print('{}: Creating attenuation map '.format(telname),
              'at {:.0f} MHz, flat earth assumption'.format(frequency.to(u.MHz).value))
        start_time_loc = time.time()
        height_profs_resc = copy.deepcopy(hprof_cache['height_profs'])
        hprof_cache['height_profs'][:] = 0.
        attenmaps_0 += [pathprof.atten_map_fast(
            frequency,
            params['temperature'],
            params['pressure'],
            telpars.height,
            h_rg,
            params['timepercent'],
            hprof_cache
        )['L_b']]
        hprof_cache['height_profs'] = height_profs_resc
        print('Time after start: {} minutes'.format((time.time() -
                                                     start_time_tot)/60.))
        print('Time for this task: {} minutes'.format((time.time() -
                                                       start_time_loc)/60.))

        print('{}: Creating stats for attenuation map'.format(telname),
              ' at {:.0f} MHz'.format(frequency.to(u.MHz).value))
        start_time_loc = time.time()
        a = attenmaps[-1]
        averatts += [averatt(attenmaps[-1], hprof_cache, **genparams)]
        print('Time after start: {} minutes'.format((time.time() -
                                                     start_time_tot)/60.))
        print('Time for this task: {} minutes'.format((time.time() -
                                                       start_time_loc)/60.))

        print('{}: Creating stats for attenuation map'.format(telname),
              ' at {:.0f} MHz,'.format(frequency.to(u.MHz).value),
              ' flat earth assumption')
        start_time_loc = time.time()
        averatts_0 += [averatt(attenmaps_0[-1], hprof_cache, **genparams)]
        print('Time after start: {} minutes'.format((time.time() -
                                                     start_time_tot)/60.))
        print('Time for this task: {} minutes'.format((time.time() -
                                                       start_time_loc)/60.))

    output['attenmaps'] = attenmaps
    output['attenmaps_0'] = attenmaps_0
    output['averatts'] = averatts
    output['averatts_0'] = averatts_0
    return output

### Batch plotting
The following function will plot the data that we generated in the
previous step, a topographical map for any frequency, and an
attenuation map and a statistical profile for a set of frequencies.

In [ ]:
def plotattdata(attdata, tellist, sites, att_lo=2, att_hi=90,
                ctics=10, ave_lo=0.0, ave_hi=100, grid=10,
                inlay=(0.3, 0.13, 0.65, 0.25), avmd=2):
    """Generate topographical and attenuation maps and attenuation profiles

    Use a dictionary of maps and other data as generated by function
    getattdata (this module) to create a set of maps. The dict attdata
    is described as output in function getattdata, tellist is a list
    of telescope properties as described in function getattdata, sites
    is a list of landmarks as described in function plot_attenmap
    (this module). The function will first plot a topographic map and
    generate a hardcopy with the name xxx_hei.pdf, where xxx is the
    short id of the telescope. For each frequency, the attenuation
    maps will be plotted, and hardcopies xxx_att_yyyy.pdf will
    generated, where xxx is the short id of the telescope, and yyyy
    the frequency in MHz. The maximum and minimum of the colour scheme
    are determined by the percentiles at percentages att_hi (maximum)
    and att_lo (minimum). The colour bar tick mark separation is given
    by the ctics parameter. Finally, circularly averaged attenuation
    profiles are plotted with a hardcopy xxx_rat_yyyy.pdf (xxx and
    yyyy same as above), see function plotaveratt (this module) for
    details. The maximum and minimum of the ordinate are determined by
    the percentiles at percentages ave_hi (maximum) and ave_lo
    (minimum), the grid separation along the ordinate by the parameter
    grid (in dB), the size of the inlay by the parameter inlay (see
    plotaveratt), and the minimum range for averaging the difference
    between conventional profiles and flat-earth profiles is given by
    the parameter avmd.

    Parameters:
    attdata (dict):      Dictionary of attenuation maps and profiles
    tellist (dict):      List of available telescopes
    sites (OrderedDict): Dictionary with site information
    att_lo (float):      Percentage of lower percentile to calculate
                         attenuation map colour scheme
    att_hi (float):      Percentage of higher percentile to calculate
                         attenuation map colour scheme
    ctics (float):       Difference between tickmarks on colour bar
    ave_lo (float):      Percentage of lower percentile to calculate
                         attenuation profile ordinate limits
    ave_hi (float):      Percentage of higher percentile to calculate
                         attenuation profile ordinate limits
    grid (float):        Grid line separation in the main viewgraph
    inlay (tuple):       Dimensions of inlay on viewgraph
    avmd (float):        Minimum radius for averaging inlay curve

    Returns:
        void
    """
    heightmap = attdata['heightmap']
    lons = attdata['lons']
    lats = attdata['lats']
    telname = attdata['telname']
    telongname = tellist[telname].name
    frequencies = attdata['frequencies']
    attenmaps = attdata['attenmaps']
    attenmaps_0 = attdata['attenmaps_0']
    averatts = attdata['averatts']
    averatts_0 = attdata['averatts_0']

    print('{}: Plotting height map'.format(telname))

    plotheightmap(heightmap, sites, lons, lats,
                  outname='{}_hei.pdf'.format(telname),
                  title='Area around {}'.format(telongname))
    for freqnum in range(len(frequencies)):
        attmap = attenmaps[freqnum]
        reshaped = attmap.reshape(attmap.shape[0]*attmap.shape[1]).value
        reshdist = attdata['distmap'].reshape(attmap.shape[0]*attmap.shape[1])
        reshaped = reshaped[reshdist < averatts[freqnum]['bin_dists'].max()]
        vmin = np.percentile(reshaped, att_lo)
        vmax = np.percentile(reshaped, att_hi)
        ymin = np.percentile(reshaped, ave_lo)
        ymax = np.percentile(reshaped, ave_hi)+5.
        freqn = frequencies[freqnum].to(u.MHz).value
        print('{}: Plotting attenuation map'.format(telname),
              'at {:.0f} MHz'.format(freqn))
        plot_attenmap(attmap, sites, lons, lats,
                      outname='{}_att_{:04.0f}.pdf'.format(telname, freqn),
                      vmin=vmin, vmax=vmax,
                      title='{} {:.0f} MHz'.format(telongname, freqn),
                      ctics=ctics)
        print('{}: Plotting average attenuation '.format(telname),
              'at {:.0f} MHz'.format(freqn))
        plotaveratt(averatts[freqnum], averatts2=averatts_0[freqnum],
                    title='{} {:.0f} MHz'.format(telongname, freqn),
                    outname='{}_rat_{:04.0f}.pdf'.format(telname, freqn),
                    ymin=ymin, ymax=ymax, grid=grid,
                    inlay=inlay, avmd=avmd)
    return

### Putting things together, a study of attenuation profiles for different telescopes and frequencies
We are now ready to generate a set of plots. Our main question is how predicted attenuation profiles differ between frequencies, depending on the location, and depending on the flat earth assumption. We choose two frequencies, for which we consider these questions, $610\,\mathrm{MHz}$, and $6650\,\mathrm{MHz}$. The chosen sites are the WSRT in The Netherlands in a rather flat terrain, and the Effelsberg radio telescope in a middle mountain range. We create very large maps, such that we can generate attenuation profiles of up to $500\,\mathrm{km}$. The time percentage (the fraction of time in which the attenuation is below the value calculated according to [ITU-R Recommendation P.452](https://www.itu.int/rec/R-REC-P.452-16-201507-I/en)) is chosen to be 2%, the statistically allowable loss for the Radio Astronomy Services. We choose the high map resolution of $3^{\prime\prime}$. We then generate the profile statistics. For each telescope for each frequency we generate and attenuation map taking geography into account and using the flat earth approximation. The profile viewgraphs show:

Black line: attenuation, 50% percentile
Green area: attenuation, between 5% perzentile and 95% perzentile
Cyan area: attenuation, enclosing minimum and maximum
Red line: flat-earth approximation

The inlay shows the difference of the main quantities and the red line (flat earth) in the same colours, while the red line shows the average of the difference between the median and the flat earth line beyond a distance of $10\,\mathrm{km}$. This is done in two steps, the time consuming calculation of the maps (it takes a long time), and the faster generation of the plots.

In [ ]:
# Characteristics of transmitters, currently height above ground
transmitters = {
    610. * u.MHz : 30 * u.m,
    6650. * u.MHz: 20 * u.m
}

# Parameters for pycraf attenuation map calculations
pars = {
    'map_size_lon': 3.0 * u.deg, 
    'map_size_lat': 3.0 * u.deg,
    'omega': 0. * u.percent,  # fraction of path over sea
    'temperature': 290. * u.K,
    'pressure': 1013. * u.hPa,
    'timepercent': 2 * u.percent,  # see P.452 for explanation
    'zone_t': pathprof.CLUTTER.UNKNOWN,
    'zone_r': pathprof.CLUTTER.UNKNOWN,
    'map_resolution': 30. * u.arcsec
}

# Parameters for circular statistics on attenuation maps
genpars = {
    'lo_perc': 5,
    'hi_perc' : 95,
#    'rmax': 500 * u.km,
#    'bins': 500
    'rmax':  200 * u.km,
    'bins': 50
}

telprocess = {
    # att_lo, att_hi, ave_lo, ave_hi, grid, ctics, inlay, avmd
    #  input to plotdata
    'Wb': (2, 90, 0.0, 100, 10, 10, (0.3, 0.13, 0.65, 0.25), 10),
    'Eb': (2, 90, 0.0, 100, 10, 20, (0.3, 0.13, 0.65, 0.25), 10)
}

plt.close()
attdata = {}
for telescope in telprocess.keys():
    attdata[telescope] = getattdata(telescope, telinfo, pars, genpars, transmitters)
    

In [ ]:
telprocess = {
    # att_lo, att_hi, ave_lo, ave_hi, grid, ctics, inlay, avmd
    #  input to plotdata
    'Wb': (2, 90, 0.0, 100, 10, 10, (0.3, 0.13, 0.65, 0.25), 80),
    'Eb': (2, 90, 0.0, 100, 10, 20, (0.3, 0.13, 0.65, 0.25), 80)
}
for telescope in telprocess.keys():
    plotattdata(attdata[telescope], telinfo, sites,
                att_lo=telprocess[telescope][0], 
                att_hi=telprocess[telescope][1], 
                ave_lo=telprocess[telescope][2], 
                ave_hi=telprocess[telescope][3], 
                grid=telprocess[telescope][4],
                ctics=telprocess[telescope][5],
                inlay=telprocess[telescope][6],                
                avmd=telprocess[telescope][7]                
               )
    

### Results
As one can see, the flat earth is a better approximation to the Westerbork telescope than Effelsberg, which is not a surprise. There is nearly no difference between a flat earth assumption and the prediction using the geographical information for the Westerbork telescope. In the far field the difference between Effelsberg using geographical data and the flat earth is about 30 dB, or a factor of 1000.